In [18]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [19]:
df = pd.read_excel('/data-dashboard/dat/HomeBridger Data Feb112025 (Do not edit).xlsx', sheet_name=None)

In [20]:
client_goal_df = df['CLIENT_GOAL__C']
program_engagement_df = df["PMDM__PROGRAMENGAGEMENT__C"]

# Create a mapping between family collective IDs and their members' races
family_race_mapping = {}

# For each row in program_engagement_df
for _, row in program_engagement_df.iterrows():
    # Get the external ID and family collective enrollment ID
    external_id = row['EXTERNAL_ID__C']
    family_id = row['EX_FAMILY_COLLECTIVE_ENROLLMENT__C']
    race = row['RACE__C']
    
    if pd.notna(race):  # Only consider non-null race values
        # If this is a family member (has a family ID)
        if pd.notna(family_id):
            # Add this member's race to the head's mapping
            if family_id in family_race_mapping:
                if race not in family_race_mapping[family_id]:
                    family_race_mapping[family_id].append(race)
            else:
                family_race_mapping[family_id] = [race]
        
        # Also add the individual's own race to their own record
        if external_id in family_race_mapping:
            if race not in family_race_mapping[external_id]:
                family_race_mapping[external_id].append(race)
        else:
            family_race_mapping[external_id] = [race]

# Convert lists of races to a single string value
for id_key, races in family_race_mapping.items():
    family_race_mapping[id_key] = '; '.join(races)

# Now create a new column with combined race information
program_engagement_df['COMBINED_RACE'] = program_engagement_df['EXTERNAL_ID__C'].map(
    lambda x: family_race_mapping.get(x, program_engagement_df.loc[program_engagement_df['EXTERNAL_ID__C'] == x, 'RACE__C'].values[0] 
              if not pd.isna(program_engagement_df.loc[program_engagement_df['EXTERNAL_ID__C'] == x, 'RACE__C']).all() else None)
)

# Now proceed with your original filtering
program_engagement_df = program_engagement_df.drop_duplicates(subset=['EXTERNAL_ID__C'], keep='first')
program_engagement_df = program_engagement_df[program_engagement_df['EX_FAMILY_COLLECTIVE_ENROLLMENT__C'].isna()]

# Merge the dataframes
merged_df = client_goal_df.merge(program_engagement_df,
                                 left_on="EXT_HOUSEHOLD__C",
                                 right_on="EXTERNAL_ID__C",
                                 how="inner")

merged_df = merged_df.dropna(axis=1, how='all')
 
# Filter by goal status
filtered_df = merged_df[merged_df["GOAL_STATUS__C"].isin(["Achieved", "Not Achieved"])]
filtered_df["GOAL_STATUS_BINARY"] = filtered_df["GOAL_STATUS__C"].map({"Achieved": 1, "Not Achieved": 0})


categorical_columns = [
   'DOMAIN__C', 'OUTCOME__C',
   'SPECIFIC_GOAL_ACTION_STEP__C', 'PMDM__STAGE__C',
   'REASON_CLOSED__C', 'LIVING_SITUATION_AT_EXIT__C',
   'LIVING_SITUATION_PRIOR_TO_ENTRY__C', 'PREGNANCY_STATUS_AT_ENROLLMENT__C',
   'LIVING_SITUATION_AT_ENTRY__C', 'SUBSIDY_AT_EXIT__C', 'COUNTY_AT_ENTRY__C',
   'COUNTY_AT_EXIT__C', 'RECORD_ORIGIN__C_y', 'TIME_TO_COMPLETE', 'COMBINED_RACE',
]


drop_columns =  ['SYSTEMMODSTAMP_x', 'GOAL_STATUS__C', 'EXT_HOUSEHOLD__C', 'EXT_CLIENT__C', 'EXTERNAL_ID__C_x', 'RECORD_HASH__C_x', 'TFC_SYNC__C_x', 'EX_ACCOUNT__C', 'SYSTEMMODSTAMP_y', 'EX_CONTACT__C', 'PMDM__APPLICATIONDATE__C', 'LAST_NAME__C', 'VERBAL_CONSENT_GIVEN_SALESFORCE_ROI__C', 'FIRST_NAME__C', 'VERBAL_CONSENT_GIVEN_HMIS_ROI__C', 'EMAIL__C', 'BIRTHDATE__C', 'VERBAL_CONSENT_GIVEN_MNPS_ROI__C', 'HMIS_ROI__C', 'HMIS_SALESFORCE__C', 'MNPS_ROI__C', 'GENDER__C',  'ETHNICITY__C', 'VETERAN_STATUS__C', 'PHONE__C', 'CLIENT_DOESN_T_KNOW__C', 'RECORD_TYPE_NAME__C', 'ENROLLED_IN_TFC_CHILDCARE__C', 'ZIPCODE_AT_EXIT__C', 'ZIPCODE_AT_ENTRY__C', 'EX_PARTNER_AGENCY__C', 'CLOSURE_DATE__C', 'CLOSURE_REASON__C', 'EXTERNAL_ID__C_y', 'CLONED_FROM_ID__C', 'RECORD_HASH__C_y', 'TFC_SYNC__C_y', 'RECORD_SOURCE__C','PMDM__ENDDATE__C','RECORD_ORIGIN__C_x','LATEST_DATE_OF_HOMELESSNESS__C','PMDM__STARTDATE__C','DATE_GOAL_ACHIEVED__C','PMDM__ROLE__C', 'RACE__C']

dropped_df = filtered_df.drop(columns=drop_columns)
dropped_df['DATE_GOAL_SET__C'] = pd.to_datetime(dropped_df['DATE_GOAL_SET__C'], errors='coerce')
dropped_df['TARGET_COMPLETION_DATE__C'] = pd.to_datetime(dropped_df['TARGET_COMPLETION_DATE__C'], errors='coerce')
# compute the time difference between the target completion date and the date the goal was set
dropped_df['TIME_TO_COMPLETE'] = (dropped_df['TARGET_COMPLETION_DATE__C'] - dropped_df['DATE_GOAL_SET__C']).dt.days
dropped_df['TIME_TO_COMPLETE'] = dropped_df['TIME_TO_COMPLETE'].fillna(0)
dropped_df['TIME_TO_COMPLETE'] = dropped_df['TIME_TO_COMPLETE'].astype(int)
# if the time to complete is negative, set it to 0
dropped_df.loc[dropped_df['TIME_TO_COMPLETE'] < 0, 'TIME_TO_COMPLETE'] = 0

dropped_df.drop(columns=['DATE_GOAL_SET__C', 'TARGET_COMPLETION_DATE__C'], inplace=True)
dropped_df.fillna('Missing', inplace=True)

/var/folders/lz/kqhzsf4s1wl78kvfdr4y184c0000gn/T/ipykernel_79710/4039803877.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["GOAL_STATUS_BINARY"] = filtered_df["GOAL_STATUS__C"].map({"Achieved": 1, "Not Achieved": 0})


In [30]:
# Define non-race values to be removed
non_race_values = [
    'Client doesn\'t know', 
    'Data not collected', 
    'Not listed', 
    'Client refused'
]

# Function to clean race strings
def clean_race_string(race_str):
    if pd.isna(race_str):
        return race_str
    
    # Split the combined race string
    races = [r.strip() for r in race_str.split(';')]
    
    # Filter out non-race values
    valid_races = [race for race in races if race not in non_race_values]
    
    # If no valid races remain, return None (will become 'Missing')
    if not valid_races:
        return None
    
    # Join the remaining valid races
    return '; '.join(valid_races)

dropped_df['COMBINED_RACE'] = dropped_df['COMBINED_RACE'].apply(clean_race_string)

In [33]:
dropped_df['RECORD_ORIGIN__C_y'].value_counts()
dropped_df.to_csv('/data-dashboard/dat/all.csv', index=False)